In [12]:
!pip install python-decouple

You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
from decouple import config

In [14]:
CLIENT_SECRET= config("CLIENT_SECRET")
CLIENT_ID = config("CLIENT_ID")
CLIENT_ACCESS_TOKEN = config("CLIENT_ACCESS_TOKEN")

In [4]:
import requests

def request_song_info(i):
    base_url = 'https://api.genius.com/songs/'+str(i)
    headers = {'Authorization': 'Bearer ' + CLIENT_ACCESS_TOKEN}
    response = requests.get(base_url, headers=headers)
    if response.json()["meta"]["status"] == 200:
        return [response.json()["response"]["song"]["title"],response.json()["response"]["song"]["url"]]
    else:
        return ["error"]



In [18]:
%%time 
[request_song_info(i) for i in range(3,8)]

CPU times: user 71.5 ms, sys: 489 µs, total: 72 ms
Wall time: 885 ms


[['Can I Live', 'https://genius.com/Jay-z-can-i-live-lyrics'],
 ['Forgive Me Father', 'https://genius.com/Fabolous-forgive-me-father-lyrics'],
 ['Down and Out', 'https://genius.com/Camron-down-and-out-lyrics'],
 ['Fly In', 'https://genius.com/Lil-wayne-fly-in-lyrics'],
 ['Lollipop (Remix)', 'https://genius.com/Lil-wayne-lollipop-remix-lyrics']]

### Putting it all together

In [19]:
from bs4 import BeautifulSoup

In [20]:
def get_links(x,i):
    return x[i]['href']
def get_table_row(side_list_table_row):
    a = side_list_table_row.find_all("span", {"class":"metadata_unit-label"})
    label = a[0].text
    links_list = side_list_table_row.find_all("a")
    links = [get_links(links_list,i) for i in range(len(links_list))]
    return label, links
def get_table_from_page(url,title):
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'html.parser')
    side_list_table = soup.find_all("div", {"class":"metadata_unit metadata_unit--table_row"} )
    return [title, url, [get_table_row(side_list_table[i]) for i in range(0,len(side_list_table)) ]]

In [21]:
def get_song_info(i):
    r = request_song_info(i)
    return r
#     if r.json()["meta"]["status"] == 200:
#         title = r.json()["response"]["song"]["title"]
#         url = r.json()["response"]["song"]["url"]
#         return get_table_from_page(url,title)
#     else:
#         return ["error"+str(i)] 


    

In [22]:
get_song_info(1)

['Killa Cam', 'https://genius.com/Camron-killa-cam-lyrics']

In [14]:
%%time
results = [get_song_info(i) for i in range(1,5) ]

AttributeError: 'list' object has no attribute 'json'

In [13]:
results

NameError: name 'results' is not defined

In [15]:
import numpy as np
def request_artist_id(i):

    base_url = 'https://api.genius.com/artists/'+str(i)
    headers = {'Authorization': 'Bearer ' + CLIENT_ACCESS_TOKEN}
    response = requests.get(base_url, headers=headers)
    if response.json()["meta"]["status"] == 200:
        name = response.json()["response"]["artist"]["name"]
        artist_id = response.json()["response"]["artist"]["id"]
        return (name, artist_id)
    else:
        return (np.nan,i)

In [16]:
r = request_artist_id(1)

In [17]:
r

("Cam'ron", 1)

In [27]:
results3 = [request_artist_id(i) for i in range(25000,35000) ]

In [29]:
import pickle

In [30]:
filename = 'artist_genius_id_3'
outfile = open(filename, 'wb')
pickle.dump(results3,outfile)
outfile.close()

In [9]:
with open('artist_genius_id', 'rb') as fp:
    artists_genius_id = pickle.load(fp)

EOFError: Ran out of input

In [ ]:
with open(target, "rb") as f:
        unpickler = pickle.Unpickler(f)

In [ ]:
def request_artist_songs_list(i):
    page = 1
    base_url = 'https://api.genius.com/artists/'+str(i)+'/songs?per_page=50&page='+str(page)+"&sort=popularity"
    headers = {'Authorization': 'Bearer ' + CLIENT_ACCESS_TOKEN}
    
    
    response = requests.get(base_url, headers=headers)
    return response

In [ ]:
%%time
r = request_artist_info(1)

In [ ]:
r.json()["response"]["songs"][1]["url"]

In [ ]:
r.json()["response"]["next_page"]